In [2]:
import pandas as pd

In [3]:
train_df = pd.read_csv("../data/train.csv")

In [4]:
train_df

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,./train/TRAIN_00000.jpg,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,./train/TRAIN_00001.jpg,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,./train/TRAIN_00002.jpg,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,./train/TRAIN_00003.jpg,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,./train/TRAIN_00004.jpg,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
42806,TRAIN_42806,./train/TRAIN_42806.jpg,1,1,1,1,1,1,1,1,1,1
42807,TRAIN_42807,./train/TRAIN_42807.jpg,1,1,1,1,1,1,1,1,1,1
42808,TRAIN_42808,./train/TRAIN_42808.jpg,1,1,1,1,1,1,1,1,1,1
42809,TRAIN_42809,./train/TRAIN_42809.jpg,1,1,1,1,1,1,1,1,1,1


In [17]:
G_id = train_df["id"][(train_df['G'] == 1) & (train_df['A'] == 0) & (train_df['B'] == 0) & (train_df['C'] == 0) & (train_df['D'] == 0) & (train_df['E'] == 0) & (train_df['F'] == 0)\
        & (train_df['H'] == 0) & (train_df['J'] == 0) & (train_df['I'] == 0)]

I_id = train_df["id"][(train_df['I'] == 1) & (train_df['A'] == 0) & (train_df['B'] == 0) & (train_df['C'] == 0) & (train_df['D'] == 0) & (train_df['E'] == 0) & (train_df['F'] == 0)\
        & (train_df['G'] == 0) & (train_df['H'] == 0) & (train_df['J'] == 0)]

J_id = train_df["id"][(train_df['J'] == 1) & (train_df['A'] == 0) & (train_df['B'] == 0) & (train_df['C'] == 0) & (train_df['D'] == 0) & (train_df['E'] == 0) & (train_df['F'] == 0)\
        & (train_df['G'] == 0) & (train_df['H'] == 0) & (train_df['I'] == 0)]


In [19]:
print(len(G_id))
print(len(I_id))
print(len(J_id))

120
48
72


In [20]:
import pandas as pd
import os
import cv2
from glob import glob
from random import randint, sample
from tqdm import tqdm
import imutils
import numpy as np

In [21]:
os.makedirs('../data/G_label', exist_ok=True)
os.makedirs('../data/I_label', exist_ok=True)
os.makedirs('../data/J_label', exist_ok=True)

In [22]:
bg_list = glob("../data/crawed_img/table/*.jpg")

In [23]:
print(len(bg_list))

15414


In [11]:
new_train_csv = pd.read_csv("../data/new_train.csv")

In [24]:
cnt = 0
number = 42809
for label,folder in [(G_id,"G_label"),(G_id,"G_label"),\
     (I_id,"I_label"),(I_id,"I_label"),(I_id,"I_label"),(I_id,"I_label"),(I_id,"I_label"),\
     (J_id,"J_label"),(J_id,"J_label"),(J_id,"J_label")]:
    img_list = [f"../data/train/{i}.jpg" for i in label]
    for img_l, background in tqdm(zip(img_list, bg_list[cnt:cnt+len(img_list)])):
        # './train\\TRAIN_00999.jpg'
        name = img_l[-15:]
        num = int(name[-9:-4])
        img = cv2.imread(img_l)
        if 31600 <= num <= 31711:
            cv2.imwrite(f"../data/H_label/{name}", img)
            continue

        thresh = 230
    
        
        img_bin = np.array(img[:, :, 0] < thresh, dtype=np.uint8) * 255
        img_blur = cv2.medianBlur(img_bin, 5)
        img_filter = img.copy()
        img_filter[img_blur == 0] = 0

        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img_blur)
        stats = np.array(stats)
        stats[0, -1] = 0
        label_idx = np.argmax(stats[:, -1])
        img_mask = labels == label_idx
        img_final = img_filter.copy()
        img_final[~img_mask] = (0, 0, 0)

        img_back = cv2.resize(cv2.imread(background), img.shape[::-1][1:])
        img_back[img_mask] = (0, 0, 0)
        img_filter += img_back

        
        with open('./new_used_name.txt', 'a') as f:
            f.write(f'{background}\n')
        cv2.imwrite(f"../data/new_merge_train/TRAIN_{number}.jpg", img_filter)
        if folder == "G_label":
            new_train_csv.loc[number] = [f"TRAIN_{number}",f"./train/TRAIN_{number}.jpg",0,0,0,0,0,0,1,0,0,0]
        elif folder == "I_label":
            new_train_csv.loc[number] = [f"TRAIN_{number}",f"./train/TRAIN_{number}.jpg",0,0,0,0,0,0,0,0,1,0]
        elif folder == "J_label":
            new_train_csv.loc[number] = [f"TRAIN_{number}",f"./train/TRAIN_{number}.jpg",0,0,0,0,0,0,0,0,0,1]
        else:
            print("!!!!!!! error!!")
        number += 1
    cnt += len(img_list)

4it [00:00, 20.40it/s]

../data/train/TRAIN_00144.jpg
../data/train/TRAIN_00145.jpg
../data/train/TRAIN_00146.jpg
../data/train/TRAIN_00147.jpg
../data/train/TRAIN_00148.jpg
../data/train/TRAIN_00149.jpg


11it [00:00, 28.47it/s]

../data/train/TRAIN_00150.jpg
../data/train/TRAIN_00151.jpg
../data/train/TRAIN_00152.jpg
../data/train/TRAIN_00153.jpg
../data/train/TRAIN_00154.jpg
../data/train/TRAIN_00155.jpg
../data/train/TRAIN_00156.jpg
../data/train/TRAIN_00157.jpg
../data/train/TRAIN_00158.jpg


20it [00:00, 30.46it/s]

...

../data/train/TRAIN_22738.jpg
../data/train/TRAIN_22739.jpg
../data/train/TRAIN_22740.jpg
../data/train/TRAIN_22741.jpg
../data/train/TRAIN_22742.jpg
../data/train/TRAIN_22743.jpg


67it [00:02, 26.18it/s]

../data/train/TRAIN_22744.jpg
../data/train/TRAIN_22745.jpg
../data/train/TRAIN_22746.jpg
../data/train/TRAIN_22747.jpg
../data/train/TRAIN_22748.jpg


72it [00:03, 21.83it/s]

../data/train/TRAIN_22749.jpg
../data/train/TRAIN_22750.jpg
../data/train/TRAIN_22751.jpg


In [ ]:
new_train_csv.to_csv("../igh_train_csv.csv")